<a href="https://colab.research.google.com/github/juoklee/ai-exploration-notebooks/blob/main/7_1_DuckDuckGo_20251012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검색기능

In [ ]:
!pip install duckduckgo-search openai langchain_community

In [ ]:
pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [ ]:
# duckduckgo를 이용한 검색 툴
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("이순신 장군이 언제 태어나셨어?")

'1 week ago - 이순신 (李舜臣, 1545년 4월 28일(윤음력 3월 8일)~1598년 12월 16일(음력 11월 19일)은 조선 중기 한국의 무신이다. 본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)이며 한성 출신이다. 문반 가문 출신으로 1576년(선조 9년) ... April 4, 2025 - 이순신 (李純信, 1553년 12월 27일 ~ 1611년 9월 2일)은 조선 중기의 왕족, 무장, 유학자이다. 임진왜란 때에 활하던 장수로 그의 상관이기도 했던 충무공 이순신 (李舜臣), 등림수 이순신 (李舜臣) 등과의 구별을 위해 무의공 이순신 , ... 1 week ago - 이정(李貞, 1511년 7월 2일 ~1583년 11월 15일)은 조선의 무신이다. 본관은 덕수(德水) 자는 희복(希復)이다. 행 병절교위. 증 순충적덕병의보조공신(純忠積德秉義補祚功臣) 대광보국(大匡輔國) 숭록대부(崇祿大夫) 의정부(議政府) ... December 25, 2024 - 이기순(1953년 12월 20일 ~ )은 대한민국의 정치인이다. 제39대 인제군수를 역임했다. 이기순은 제5회 전국동시지방선거를 앞두고 지역 주민들에게 물품을 제공한 혐의를 받아 검찰에 기소되었다. 이에 대법원은 이기순에게 ... 3 weeks ago - 명량 해전(鳴梁海戰) 또는 명량 대첩(鳴梁大捷)은 1597년(선조 30) 음력 9월 16일(양력 10월 26일) 정유재란 당시 조선 수군이 이순신 장군의 지휘 아래 일본 수군의 대규모 함대를 상대로 승리를 거둔 전투이다.'

# 유튜브 영상 검색

In [ ]:
!pip install --upgrade --quiet youtube_search

In [ ]:
# 검색어 관련 영상 검색
from langchain_community.tools import YouTubeSearchTool
tool = YouTubeSearchTool()
tool.run("비트코인")

"['https://www.youtube.com/watch?v=SSaF_Ropujc&pp=ygUM67mE7Yq47L2U7J24', 'https://www.youtube.com/watch?v=tuH4fCQNdco&pp=ygUM67mE7Yq47L2U7J24']"

In [ ]:
!pip install --upgrade --quiet youtube-transcript-api pytube faiss-cpu tiktoken langchain langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.2 MB/s eta 0:00:00


In [ ]:

# 검색어 관련 영상 내용 요약 까지

# From: https://towardsdatascience.com/getting-started-with-langchain-a-beginners-guide-to-building-llm-powered-applications-95fc8898732c
from langchain.document_loaders import YoutubeLoader # 유튜브 로더
from langchain.embeddings import OpenAIEmbeddings    # 임베딩용
from langchain.vectorstores import FAISS             # 벡터 스토어 종류
from langchain.chains import RetrievalQA             # 질문 답변 툴
from langchain_openai import ChatOpenAI              # OpenAI 쓸 수 있는 툴
from pprint import pprint                            # 아웃풋 프린트 툴 (필요 없음)
from pickle import load

# set key
import os, openai
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai')
openai.api_key = os.getenv("OPENAI_API_KEY")

# embedding
embeddings = OpenAIEmbeddings()

# 자막 내용 가져오기
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=HsonXuJs8-s")
docs = loader.load()

# 벡터스토어 만들기
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

# 언어모델
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# 질문 답변
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

# 질문 물어보기
query = "cold shower 하면 좋은 점? 한국말로 답해"
result = qa({"query": query})
print(result)


/tmp/ipython-input-4140752369.py:43: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


{'query': 'cold shower 하면 좋은 점? 한국말로 답해',
 'result': '차가운 샤워가 건강에 좋다는 주장은 여전히 회색 영역입니다. 그러나 일부 연구에서는 차가운 샤워가 몇 가지 이점을 가질 '
           '수 있다는 약한 증거를 제시하고 있습니다.\n'
           '\n'
           '예를 들어, 2016년에 PLOS One에서 발표된 연구에서는 아침에 차가운 샤워를 하면 직장에서 병가를 쓰는 일수가 '
           '줄어들지 확인하려고 했습니다. 이 연구에서는 2300명 이상의 참가자가 30일 동안 따뜻한 샤워를 하거나, 마지막에 '
           '30, 60, 90초 동안 차가워지는 따뜻한 샤워를 했습니다. 결과적으로, 차가운 샤워 그룹의 참가자들은 따뜻한 샤워 '
           '그룹보다 평균적으로 29% 덜 병가를 냈습니다. 이는 대략 한 달에 병가 하루를 덜 쓴 것과 같습니다.\n'
           '\n'
           '또한, 많은 사람들이 차가운 샤워가 기분을 향상시킬 수 있다고 주장합니다. 그러나 이에 대한 연구 결과는 더 약합니다. '
           '2008년에 Medical Hypothesis에서 발표된 연구에서는 차가운 물이 교감신경계를 활성화시켜 심장 박동을 '
           '빠르게 하고 엔돌핀을 분비시켜 기분을 향상시킬 것이라는 가설을 세웠습니다. 그러나 이 연구는 데이터 포인트가 세 '
           '개뿐이었고, 그 중 하나는 논문의 저자였으며, 그들 중 어느 누구도 우울증을 진단할 수 있는 증상을 가지고 있지 '
           '않았습니다.\n'
           '\n'
           '따라서, 차가운 샤워가 건강에 좋다는 주장은 아직 확실하지 않습니다. 짧은 차가운 샤워가 해를 끼치지는 않을 것이지만, '
           '실제적인 건강 효과에 대한 주장을 하기 전에 더 많은 연구가 필요합니다.',
 'sour

In [ ]:
pprint(result['result'])

('차가운 샤워가 건강에 좋다는 주장은 여전히 회색 영역입니다. 그러나 일부 연구에서는 차가운 샤워가 몇 가지 이점을 가질 수 있다는 약한 '
 '증거를 제시하고 있습니다.\n'
 '\n'
 '예를 들어, 2016년에 PLOS One에서 발표된 연구에서는 아침에 차가운 샤워를 하면 직장에서 병가를 쓰는 일수가 줄어들지 확인하려고 '
 '했습니다. 이 연구에서는 2300명 이상의 참가자가 30일 동안 따뜻한 샤워를 하거나, 마지막에 30, 60, 90초 동안 차가워지는 '
 '따뜻한 샤워를 했습니다. 결과적으로, 차가운 샤워 그룹의 참가자들은 따뜻한 샤워 그룹보다 평균적으로 29% 덜 병가를 냈습니다. 이는 '
 '대략 한 달에 병가 하루를 덜 쓴 것과 같습니다.\n'
 '\n'
 '또한, 많은 사람들이 차가운 샤워가 기분을 향상시킬 수 있다고 주장합니다. 그러나 이에 대한 연구 결과는 더 약합니다. 2008년에 '
 'Medical Hypothesis에서 발표된 연구에서는 차가운 물이 교감신경계를 활성화시켜 심장 박동을 빠르게 하고 엔돌핀을 분비시켜 '
 '기분을 향상시킬 것이라는 가설을 세웠습니다. 그러나 이 연구는 데이터 포인트가 세 개뿐이었고, 그 중 하나는 논문의 저자였으며, 그들 중 '
 '어느 누구도 우울증을 진단할 수 있는 증상을 가지고 있지 않았습니다.\n'
 '\n'
 '따라서, 차가운 샤워가 건강에 좋다는 주장은 아직 확실하지 않습니다. 짧은 차가운 샤워가 해를 끼치지는 않을 것이지만, 실제적인 건강 '
 '효과에 대한 주장을 하기 전에 더 많은 연구가 필요합니다.')


In [ ]:
# 로딩을 한국어로
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=4-cmJPA2Klg", language='ko')
documents = loader.load()

# 벡터스토어 인덱스 생성
db = FAISS.from_documents(documents, embeddings)
retriever = db.as_retriever()
llm = ChatOpenAI(model_name="gpt-4")

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

query = "내용 200자로 요약해줘"
result = qa({"query": query})

pprint(result['result'])


('40대는 경제활동 인구가 9.5% 줄고, 성과 구직 활동도 하지 않는 시어스 민구가 10년간 8만 명이 늘었습니다. 여성 층에서는 '
 '40대가 처음으로 30대보다 많이 경력이 단절되는 경우가 많았습니다. 이에 따라, 직무와 성과 중심의 임금체계 개편과 노동시장 재진입이 '
 '수월하도록 제도적 정비가 필요하다는 의견이 제기되었습니다. 최근로의 자립 지원을 위해 노동시장 변화에 대응할 수 있는 리스킬링과 업스킬링 '
 '훈련이 필요하며, 40대의 고용 문제를 해결하기 위해 복지 정책의 범위를 넓혀야 한다는 것입니다.')
